In [41]:
%pip install psycopg2-binary
%pip install pandas
%pip install nltk
%pip install transformers
%pip install scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import psycopg2
import nltk
import time
from nltk.corpus import stopwords
from transformers import pipeline

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [42]:
# Baixar as stop words da NLTK
nltk.download('stopwords')
stop_words = stopwords.words('portuguese')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\andre\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [43]:
db_config = {
    'dbname': 'diariorepublica',
    'user': 'postgres',
    'password': '1597535',
    'host': 'localhost',
    'port': '5432'
}

In [44]:
class DocumentRetrieval:
    def __init__(self, db_config):
        self.conn = psycopg2.connect(**db_config)
        self.cursor = self.conn.cursor()
        self.vectorizer = TfidfVectorizer(stop_words=stop_words)
        self.df = self.load_documents()
        self.tfidf_matrix = self.vectorizer.fit_transform(self.df['notes'])

    def load_documents(self):
        self.cursor.execute("SELECT id, notes FROM dreapp_document")
        rows = self.cursor.fetchall()
        return pd.DataFrame(rows, columns=['id', 'notes'])

    def search(self, query):
        query_vec = self.vectorizer.transform([query])
        similarities = cosine_similarity(query_vec, self.tfidf_matrix).flatten()
        related_docs_indices = similarities.argsort()[::-1]
        return self.df.iloc[related_docs_indices]

    def get_full_text(self, document_id):
        self.cursor.execute("SELECT html_text FROM dreapp_documenttext WHERE document_id=%s", (document_id,))
        result = self.cursor.fetchone()
        return result[0] if result else None

In [45]:
class QASystem:
    def __init__(self):
        self.qa_model = pipeline("question-answering", model="isel-nlp-mp05/bert-base-portuguese-cased-finetuned-squadv1-pt")

    def answer_question(self, question, context):
        start_time = time.time()
        response = self.qa_model(question=question, context=context)
        end_time = time.time()
        execution_time = end_time - start_time
        print(f"Execution time: {execution_time} seconds")
        return response['answer']

In [46]:
class QAProcessor:
    def __init__(self, db_config):
        self.document_retrieval = DocumentRetrieval(db_config)
        self.qa_system = QASystem()

    def process_query(self, query, question):
        # Retrieve relevant documents
        result = self.document_retrieval.search(query)
        document_ids = result['id'][:10]  # Get top 10 documents

        # Fetch full texts
        texts = [(doc_id, self.document_retrieval.get_full_text(doc_id)) for doc_id in document_ids]

        # Identify the most relevant document for the question
        question_vec = self.document_retrieval.vectorizer.transform([question])
        similarities = [
            (doc_id, cosine_similarity(question_vec, self.document_retrieval.vectorizer.transform([text])).flatten()[0])
            for doc_id, text in texts if text
        ]
        most_relevant_doc_id, _ = max(similarities, key=lambda x: x[1])
        print(f'Most relevant doc ID: {most_relevant_doc_id}')

        # Get the full text of the most relevant document
        full_text = self.document_retrieval.get_full_text(most_relevant_doc_id)

        # Answer the question using the context of the most relevant document
        answer = self.qa_system.answer_question(question, full_text)
        return answer

In [47]:
qa_processor = QAProcessor(db_config)

In [48]:
query = input("Introduz um tópico: ")

In [49]:
question = input("Introduz uma pergunta sobre esse tópico: ")

In [50]:
answer = qa_processor.process_query(query, question)
print(answer)

Most relevant doc ID: 5038633
Execution time: 85.78383946418762 seconds
medidas de combate à pandemia
